In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn

from torch.optim import SGD
from torch.utils.data import DataLoader

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt

import warnings 

warnings.filterwarnings('ignore')

## Överblick

**Nu ska vi träna Neurala Nätverk och få lite praktiskt erfarenhet av Hyperparameters**

Vi kommer bygga och träna Neurala Nätverk för att lösa regressionsproblem.

**GRAFIKKORTSACCELERATION**

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Available device: {device}")

Available device: cuda


**DATA**

Ladda in data & delta upp i train- och testsplit

In [3]:
iris = load_iris()
  
x_train, x_test, y_train, y_test = train_test_split(iris.data,
                                                    iris.target,
                                                    test_size=0.1)

In [22]:
for k, v in iris.items():
    print(k)

data
target
frame
target_names
DESCR
feature_names
filename
data_module


In [23]:
iris['feature_names']

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

**Uppgift 1)**

Utforska & säkerställ att du förstår datan. Du kan läsa mer här:

https://archive.ics.uci.edu/dataset/53/iris

**Omvandla dataformat till tensorer**

In [18]:
# OBS, lägg märkte till att targets omvandlas till Long-format. Detta krävs för att vår loss-funktion korrekt ska mappa output y_pred till vår y_true nedan

x_train = torch.from_numpy(x_train).type(torch.FloatTensor)
y_train = torch.from_numpy(y_train).type(torch.LongTensor) 

x_test = torch.from_numpy(x_test).type(torch.FloatTensor)
y_test = torch.from_numpy(y_test).type(torch.LongTensor)

In [16]:
torch.from_numpy(y_train).dtype

torch.int64

In [12]:
torch.from_numpy(y_train).type(torch.FloatTensor)

tensor([1., 1., 2., 2., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 2., 0., 0.,
        0., 0., 2., 1., 2., 0., 0., 1., 1., 0., 0., 0., 1., 0., 2., 2., 2., 0.,
        2., 0., 1., 1., 0., 0., 2., 2., 1., 1., 1., 1., 0., 0., 1., 1., 2., 1.,
        0., 2., 1., 0., 1., 2., 1., 1., 1., 0., 0., 2., 2., 1., 1., 2., 2., 1.,
        0., 1., 1., 0., 2., 0., 2., 1., 2., 1., 1., 0., 1., 0., 0., 2., 1., 0.,
        0., 0., 1., 2., 2., 2., 2., 2., 2., 1., 0., 2., 1., 1., 0., 2., 1., 0.,
        2., 1., 2., 1., 2., 2., 2., 1., 0., 1., 0., 0., 0., 2., 0., 2., 0., 2.,
        1., 0., 2., 2., 0., 1., 2., 2., 0.])

In [17]:
torch.from_numpy(y_train).type(torch.LongTensor).dtype

torch.int64

In [23]:
x_train[0]

tensor([-0.3722, -1.0371,  0.4094,  0.0310], device='cuda:0')

**Normalisara datan**

In [19]:

# OBS, lägg märkte till att statistiken räknas ut endast på train-split, och appliceras sedan på både train- och test.

# medelvärde
mean = x_train.mean(dim=0)

# standardavvikelse
std = x_train.std(dim=0)

x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

**Lägg in GPU-support (om tillgänglig) samt paketera training- och testdatan**

In [20]:
x_train = x_train.to(device)
y_train = y_train.to(device)

x_test = x_test.to(device)
y_test = y_test.to(device)

training_set = list(zip(x_train, y_train))             
test_set = list(zip(x_test, y_test))                  

**Uppgift 2)**

Skapa ett klass NeuralNetwork, som du ska använda för multi-class classificering.

Klassen ska ta två argument, input_dim & output_dim.

Tips: datan är inte så komplicerad, så börja med ett nätverk med relativt liten kapacitet.

Tips: kolla föreläsningsvideon för att se hur man applicerar softmax

In [ ]:
input_size = 3
output_size = 3

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(NeuralNetwork, self).__init__()
        
        # TODO: definiera lager att initiera
        self.fc1 = nn.Linear(input_size, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear
        
    def forward(self, x):
        
        # TODO: Definiera forward()-metoden

        return x

In [ ]:
input_size = 4   # spelar ej roll, beror på hur många features                   
output_size = 3  # avgör antalet klasser

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_dim, 4)
        self.layer2 = nn.Linear(4, 4)
        self.layer3 = nn.Linear(4, output_dim)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.softmax(self.layer3(x))
        return x
    
model = NeuralNetwork(input_size, output_size)

**Uppgift 3)**

Ange lämplig värden för

input_size & output_size

In [ ]:
input_size =
output_size = 

**Uppgift 4)**

Träna nu din modell för på epochs & learning rate. 

Vad verkar funka bra?

Hur ser loss-kurvorna ut? Hur tolkar du dem?

**Notera att vi lagt till ytterligare en plot under plotten för loss-kurvorna. Vad signifierar den?**

In [ ]:
batch_size = 16

epochs = 500
learning_rate = 0.05

In [ ]:
# ----------------------------------------------------------------------------------------------------------------
#    initera modell, loss_function, optimizer & dataloader


model = NeuralNetwork(input_size, output_size)
model = model.to(device)                                  # förbered modellen för GPU (om tillgänglig)


optimizer = SGD(model.parameters(), lr = learning_rate)
loss_function = torch.nn.CrossEntropyLoss()               # OBS, CrossEntropyLoss!

train_dataloader = DataLoader(training_set,                 
                              batch_size = batch_size,       
                              shuffle=True)


# ----------------------------------------------------------------------------------------------------------------
#    träna



batch_train_losses = []

epoch_train_losses = []
epoch_evaluation_losses = []
epoch_evaluation_accuracies = []

for epoch in range(epochs):
    
    model.train()

    running_loss = 0
    
    for batch in train_dataloader:
        
        y_true = batch[1]
        input_features = batch[0]
        
        y_pred=model(input_features)
        loss=loss_function(y_pred, y_true)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        batch_loss = loss.item()
        batch_train_losses.append(batch_loss)
    
    epoch_average_loss = np.average(batch_train_losses[-len(train_dataloader):])
    epoch_train_losses.append(epoch_average_loss)

# ----------------------------------------------------------------------------------------------------------------
#   evalueringssektion 

    #model.eval()
    
    y_true = y_test
    input_features = x_test
    
    y_pred = model(input_features)
    loss = loss_function(y_pred, y_true)
    
    evaluation_loss = loss.item()
    epoch_evaluation_losses.append(evaluation_loss)

# ----------------------------------------------------------------------------------------------------------------
#   evalueringssektion del 2 - samla in accuracy på test set


    _, predicted_class = torch.max(y_pred, dim=1)
    accuracy = (predicted_class == y_true).float().mean().item()
    epoch_evaluation_accuracies.append(accuracy)

# ----------------------------------------------------------------------------------------------------------------
#   plotta resultat 

plt.plot(epoch_train_losses, label = 'train loss')
plt.plot(epoch_evaluation_losses, label = 'test loss', color='orange')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()

plt.show()

plt.plot(epoch_evaluation_accuracies, label = 'test accuracy', color='orange')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend()

plt.show()

**Uppgift 5)**

Försök få så bra prestanda som möjligt genom att ändra nätverkets kapacitet. Börja med att justera antal lager och antal neuroner i varje lager.

Får du ko

**Uppgift 6)**

Om du anser det nödvändigt, lägg även till Dropout